#### **Partie 1: Importation des Bibliothèques et Configuration Initiale**

In [ ]:
# Importer les bibliothèques nécessaires
from google.colab import drive
drive.mount('/content/drive')

import shutil
import torch
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
import pandas as pd


**Explication:**
- **google.colab.drive** : Monter Google Drive pour accéder aux fichiers stockés sur Drive.
- **shutil** : Pour copier des fichiers et dossiers.
- **torch** : Bibliothèque pour les opérations sur le GPU avec PyTorch.
- **pandas** : Manipule les données sous forme de DataFrame.
- **re** : Utilisé pour le traitement de texte.
- **sklearn.model_selection.train_test_split** : Diviser les données en ensembles d'entraînement et de validation.
- **transformers** : Fournit des outils pour le modèle T5 et la gestion de l'entraînement.
- **datasets** : Permet de charger des datasets à partir de fichiers CSV.

---


#### **Partie 2: Définition des Constantes et Chargement des Données**

In [ ]:
# Définir les constantes
MODEL = 't5-small'
BATCH_SIZE = 16
EPOCHS = 2
OUT_DIR = 'PEFT_v2'
MAX_LENGTH = 256
NUM_PROCS = 1  # Utiliser un seul processus pour éviter les problèmes avec os.fork()
modell = '/content/drive/My Drive/PEFT_1/PEFT_v1/checkpoint-10000'
token = '/content/drive/My Drive/PEFT_1/PEFT_v1/'

# Charger les données à partir du fichier CSV
csv_file = '/content/drive/My Drive/ahmed/en_wiki_v1.csv'
df = pd.read_csv(csv_file)

# Diviser les données en ensembles d'entraînement et de validation
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# Sauvegarder les ensembles en fichiers CSV
output_file1 = 'train.csv'
output_file2 = 'val.csv'
df_train.to_csv(output_file1, index=False)
df_val.to_csv(output_file2, index=False)


**Explication:**
- **MODEL**, **BATCH_SIZE**, **EPOCHS**, etc. : Définir les hyperparamètres pour le modèle et l'entraînement.
- **modell** et **token** : Chemins vers le modèle pré-entraîné et le tokenizer sauvegardé.
- **csv_file** : Chemin vers le fichier CSV contenant les données.
- Les données sont chargées, divisées en ensembles d'entraînement et de validation, puis sauvegardées en fichiers CSV.

---

#### **Partie 3: Chargement des Datasets et Prétraitement**

In [ ]:

# Charger les datasets à partir des fichiers CSV
dataset_train = load_dataset('csv', data_files='train.csv', split='train')
dataset_valid = load_dataset('csv', data_files='val.csv', split='train')

# Initialiser le tokenizer
tokenizer = T5Tokenizer.from_pretrained(token)

def preprocess_function(examples):
    inputs = [f" paraphrase : {original}" for (original) in zip(examples['original'])]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    cleaned_tag = examples['paraphrase']
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            cleaned_tag,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Appliquer la fonction de prétraitement aux datasets
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS  # Utiliser un seul processus
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS  # Utiliser un seul processus
)



**Explication:**
- Les datasets CSV sont chargés en utilisant `load_dataset`.
- Le tokenizer T5 est initialisé avec le chemin du tokenizer pré-entraîné.
- La fonction `preprocess_function` prépare les entrées et les étiquettes pour le modèle.
- Le prétraitement est appliqué aux ensembles d'entraînement et de validation.

---

#### **Partie 4: Initialisation du Modèle et Configuration de PEFT**

In [ ]:

# Initialiser le modèle T5
model = T5ForConditionalGeneration.from_pretrained(modell)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from peft import LoraConfig, get_peft_model, TaskType

# Configurer Lora
lora_config = LoraConfig(
    r=32,  # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM  # FLAN-T5
)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# Appliquer la configuration Lora au modèle
peft_model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))



**Explication:**
- Le modèle T5 est chargé depuis le chemin spécifié et déplacé vers le GPU si disponible.
- La configuration Lora est définie pour l'entraînement avec des paramètres spécifiques comme le rang et le taux de dropout.
- La fonction `print_number_of_trainable_model_parameters` calcule et affiche le nombre de paramètres entraînables du modèle.



#### **Partie 5: Configuration des Arguments d'Entraînement et Entraînement du Modèle**

In [ ]:

# Définir les arguments d'entraînement
peft_training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',  # Utilisation de `evaluation_strategy` au lieu de `eval_strategy`
    save_steps=5000,
    eval_steps=5000,  # Évalue la perte de validation tous les 50 pas
    load_best_model_at_end=True,
    save_total_limit=5,
    report_to='tensorboard',
    learning_rate=0.001,
    fp16=True,
    dataloader_num_workers=0  # Réduire le nombre de travailleurs
)

# Entraîner le modèle avec PEFT
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,  # Dataset de validation (test)
)

history = peft_trainer.train()


**Explication:**
- Les arguments pour l'entraînement sont configurés, y compris le nombre d'époques, la taille des lots, et la stratégie de sauvegarde.
- Un `Trainer` est créé pour entraîner le modèle avec la configuration PEFT.

---

#### **Partie 6: Sauvegarde du Tokenizer et du Modèle**

In [ ]:

# Sauvegarder le tokenizer
tokenizer.save_pretrained(OUT_DIR)

# Copier les résultats vers Google Drive
source_folder = f'./{OUT_DIR}'
target_folder = f'/content/drive/My Drive/PEFT_2/{OUT_DIR}'
shutil.copytree(source_folder , target_folder)



**Explication:**
- Le tokenizer est sauvegardé dans le répertoire spécifié.
- Les résultats de l'entraînement sont copiés dans Google Drive pour la sauvegarde.

---


#### **Partie 7: Chargement du Modèle Sauvegardé et Correction du Texte**

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Charger le modèle et le tokenizer sauvegardés
model_path = '/content/drive/My Drive/PEFT_1/PEFT_v1/checkpoint-10000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('/content/drive/My Drive/PEFT_1/PEFT_v1/')

def do_correction(text, model, tokenizer):
    input_text = f" paraphrase : {text}"
    inputs = tokenizer.encode(
        input_text,
        return_tensors='pt',
        max_length=256,
        padding='max_length',
        truncation=True
    )

    # Générer la phrase corrigée
    corrected_ids = model.generate(
        inputs,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )

    # Décoder la phrase corrigée
    corrected_sentence = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    return corrected_sentence

text = """
Thunderstorms and strong gusts of wind with dust blowers on Sunday
"""

print(do_correction(text, model, tokenizer))


**Explication:**
- Le modèle et le tokenizer sauvegardés sont rechargés.
- La fonction `do_correction` utilise le modèle pour générer une paraphrase du texte d'entrée.
- Le texte est passé à la fonction et le résultat est affiché.

---